In [7]:
import os
import torch
import pytorch_lightning as pl
from my_dataset.dataloader import BrainDataModule
from my_lightning_module.segmenter import SegModel, ModelEnum, MetricTracker
from pytorch_lightning.callbacks import ModelCheckpoint
import numpy as np

In [ ]:
nums_folds = 5
split_seed = 12345
for model_enum in ModelEnum:
    results = []
    # if model_enum != ModelEnum.UNET:
        
    for k in range(1, nums_folds):
        # k=1
        segmenter = SegModel(model_enum=model_enum)

        # saves top-K checkpoints based on "val_loss" metric
        checkpoint_callback = ModelCheckpoint(
            save_top_k=3,
            monitor="val_loss",
            mode="min",
            filename=f"{model_enum.name}"+"-{epoch:02d}-{val_loss:.2f}",
        )
        
        metric_tracker_callback = MetricTracker()
        
        trainer = pl.Trainer(
            accelerator='gpu',
            callbacks=[checkpoint_callback, metric_tracker_callback],
            max_epochs=30
        ) # , fast_dev_run=True

        data_dir = '/media/idham/SpaceX/333_workspace/my_lab/stacked_img_msk_data_dirs.npy'
        datamodule = BrainDataModule(data_dir=data_dir, k=k, num_splits=nums_folds, split_seed=split_seed, batch_size=40, num_workers=os.cpu_count())
        datamodule.prepare_data()
        datamodule.setup()

        trainer.fit(model=segmenter, datamodule=datamodule)
        
        val_loss = trainer.logged_metrics['val_loss'].item()
        score = round((1-val_loss)*100,2)        
        results.append(score)
        
        np.save(f"metric_collection_{nums_folds}_fold_k_{k}_{model_enum.name}", metric_tracker_callback.collection)

        # score = round(sum(results) / 1, 2)
        # np.save(f"score_{nums_folds}_fold_{model_enum.name}", score)

    score = round(sum(results) / nums_folds, 2)
    np.save(f"score_{nums_folds}_fold_{model_enum.name}", score)